# 1. Import and Install Dependencies

In [1]:
import pandas as pd
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
%matplotlib inline
import time
import mediapipe as mp

In [2]:
#!python -V  # 3.10.4

# Keypoints using MP Holistic

In [3]:
mp_holistic = mp.solutions.holistic # Holistic model - make our detection 
mp_drawing = mp.solutions.drawing_utils # Drawing utilities - make our drawings

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [7]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections # (FACEMESH_CONTOURS, FACEMESH_LIPS, FACEMESH_FACE_OVAL)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [8]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(0,0,255), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(0,0,255), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=2), 
                             mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=2), 
                             mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(0,0,255), thickness=2, circle_radius=2), 
                             mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2)
                             ) 

# Extract Keypoint Values

In [10]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [65]:
# add folders to the existing sign folders

# Load Model

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [14]:
from tensorflow.keras.models import load_model
model = load_model('Demo Models/Model_13ws_4p_5fps_new.h5')

In [86]:
# import keras
# new_model = keras.models.load_model('Demo Models/Model_10ws_4p_2.h5')

# Define Model Parameters

In [15]:
actions = np.array(['NoSign','hello','you','work','where','how','your','day','b','o','me','live','university'])
sequence_length = 5

# Test in Real Time

In [16]:
colors = [(245,117,16)]*len(actions)
#colors = [(245,117,16), (117,245,16), (16,117,245),(16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,50+num*30), (int(prob*100), 80+num*30), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 75+num*30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [3]:
# cap = cv2.VideoCapture(0)
# # Set mediapipe model 
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#     while cap.isOpened():

#         # Read feed
#         ret, frame = cap.read()
#         #frame = cv2.resize(frame, (512, 384))

#         # Make detections
#         image, results = mediapipe_detection(frame, holistic)
#         #print(results)
        
#         #draw_landmarks(image, results)
#         # Draw landmarks
#         draw_styled_landmarks(image, results)
        
#         #image = video_rescaling(image, 1.30)

#         # Show to screen
#         cv2.imshow('OpenCV Feed', image)

#         # Break gracefully
#         if cv2.waitKey(10) & 0xFF == ord('q'):
#             break
#     cap.release()
#     cv2.destroyAllWindows()

## Realtime prediction

In [24]:
#actions = np.array(['NoSign','please', 'yourewelcome', 'sorry', 'hello', 'thanks', 'iloveyou'])

# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.80

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        #print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-sequence_length:]
        
        if len(sequence) == sequence_length:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            #sentence.append(actions[np.argmax(res)])
                            vals, counts = np.unique(predictions[-8:], return_counts=True)
                            if vals[0]==predictions[-1] and counts[0]>5:
                                sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 39ms/step
NoSign
1/1 [==============================] - 0s 194ms/step
NoSign
1/1 [==============================] - 0s 29ms/step
day
1/1 [==============================] - 0s 29ms/step
day
1/1 [==============================] - 0s 35ms/step
me
1/1 [==============================] - 0s 47ms/step
me
1/1 [==============================] - 0s 40ms/step
me
1/1 [==============================] - 0s 33ms/step
me
1/1 [==============================] - 0s 41ms/step
me
1/1 [==============================] - 0s 33ms/step
me
1/1 [==============================] - 0s 30ms/step
live
1/1 [==============================] - 0s 39ms/step
NoSign
1/1 [==============================] - 0s 32ms/step
NoSign
1/1 [==============================] - 0s 29ms/step
NoSign
1/1 [==============================] - 0s 32ms/step
NoSign
1/1 [==============================] - 0s 32ms/step
NoSign
1/1 [==============================] - 0s 34ms/step
NoSign
1/1 [=======================

1/1 [==============================] - 0s 30ms/step
NoSign
1/1 [==============================] - 0s 29ms/step
NoSign
1/1 [==============================] - 0s 29ms/step
NoSign
1/1 [==============================] - 0s 29ms/step
NoSign
1/1 [==============================] - 0s 42ms/step
NoSign
1/1 [==============================] - 0s 40ms/step
day
1/1 [==============================] - 0s 32ms/step
day
1/1 [==============================] - 0s 27ms/step
day
1/1 [==============================] - 0s 29ms/step
day
1/1 [==============================] - 0s 31ms/step
me
1/1 [==============================] - 0s 39ms/step
day
1/1 [==============================] - 0s 35ms/step
me
1/1 [==============================] - 0s 34ms/step
me
1/1 [==============================] - 0s 40ms/step
me
1/1 [==============================] - 0s 35ms/step
me
1/1 [==============================] - 0s 35ms/step
hello
1/1 [==============================] - 0s 32ms/step
hello
1/1 [=============================

1/1 [==============================] - 0s 42ms/step
NoSign
1/1 [==============================] - 0s 28ms/step
NoSign
1/1 [==============================] - 0s 38ms/step
NoSign
1/1 [==============================] - 0s 32ms/step
NoSign
1/1 [==============================] - 0s 32ms/step
NoSign
1/1 [==============================] - 0s 35ms/step
o
1/1 [==============================] - 0s 34ms/step
o
1/1 [==============================] - 0s 29ms/step
o
1/1 [==============================] - 0s 38ms/step
o
1/1 [==============================] - 0s 36ms/step
NoSign
1/1 [==============================] - 0s 40ms/step
NoSign
1/1 [==============================] - 0s 40ms/step
NoSign
1/1 [==============================] - 0s 36ms/step
NoSign
1/1 [==============================] - 0s 30ms/step
NoSign
1/1 [==============================] - 0s 46ms/step
NoSign
1/1 [==============================] - 0s 45ms/step
live
1/1 [==============================] - 0s 41ms/step
day
1/1 [=================

NoSign
1/1 [==============================] - 0s 26ms/step
NoSign
1/1 [==============================] - 0s 28ms/step
NoSign
1/1 [==============================] - 0s 33ms/step
NoSign
1/1 [==============================] - 0s 27ms/step
NoSign
1/1 [==============================] - 0s 28ms/step
NoSign
1/1 [==============================] - 0s 31ms/step
NoSign
1/1 [==============================] - 0s 28ms/step
NoSign
1/1 [==============================] - 0s 28ms/step
NoSign
1/1 [==============================] - 0s 33ms/step
NoSign
1/1 [==============================] - 0s 34ms/step
NoSign
1/1 [==============================] - 0s 33ms/step
NoSign
1/1 [==============================] - 0s 31ms/step
day
1/1 [==============================] - 0s 38ms/step
day
1/1 [==============================] - 0s 29ms/step
NoSign
1/1 [==============================] - 0s 36ms/step
NoSign
1/1 [==============================] - 0s 28ms/step
NoSign
1/1 [==============================] - 0s 32ms/step
NoS

1/1 [==============================] - 0s 31ms/step
NoSign
1/1 [==============================] - 0s 32ms/step
NoSign
1/1 [==============================] - 0s 35ms/step
NoSign
1/1 [==============================] - 0s 36ms/step
you
1/1 [==============================] - 0s 38ms/step
you
1/1 [==============================] - 0s 39ms/step
you
1/1 [==============================] - 0s 29ms/step
you
1/1 [==============================] - 0s 30ms/step
you
1/1 [==============================] - 0s 33ms/step
day
1/1 [==============================] - 0s 26ms/step
me
1/1 [==============================] - 0s 30ms/step
day
1/1 [==============================] - 0s 32ms/step
day
1/1 [==============================] - 0s 36ms/step
day
1/1 [==============================] - 0s 32ms/step
day
1/1 [==============================] - 0s 34ms/step
day
1/1 [==============================] - 0s 39ms/step
day
1/1 [==============================] - 0s 39ms/step
day
1/1 [==============================] - 0

NoSign
1/1 [==============================] - 0s 32ms/step
NoSign
1/1 [==============================] - 0s 32ms/step
NoSign
1/1 [==============================] - 0s 28ms/step
NoSign
1/1 [==============================] - 0s 29ms/step
NoSign
1/1 [==============================] - 0s 45ms/step
NoSign
1/1 [==============================] - 0s 28ms/step
NoSign
1/1 [==============================] - 0s 30ms/step
NoSign
1/1 [==============================] - 0s 33ms/step
NoSign
1/1 [==============================] - 0s 34ms/step
NoSign
1/1 [==============================] - 0s 31ms/step
NoSign
1/1 [==============================] - 0s 31ms/step
NoSign
1/1 [==============================] - 0s 28ms/step
NoSign
1/1 [==============================] - 0s 28ms/step
NoSign
1/1 [==============================] - 0s 29ms/step
NoSign
1/1 [==============================] - 0s 28ms/step
NoSign
1/1 [==============================] - 0s 33ms/step
NoSign
1/1 [==============================] - 0s 36ms/st

1/1 [==============================] - 0s 26ms/step
how
1/1 [==============================] - 0s 39ms/step
how
1/1 [==============================] - 0s 39ms/step
how
1/1 [==============================] - 0s 36ms/step
how
1/1 [==============================] - 0s 37ms/step
how
1/1 [==============================] - 0s 37ms/step
me
1/1 [==============================] - 0s 34ms/step
me
1/1 [==============================] - 0s 36ms/step
day
1/1 [==============================] - 0s 36ms/step
o
1/1 [==============================] - 0s 36ms/step
how
1/1 [==============================] - 0s 32ms/step
how
1/1 [==============================] - 0s 34ms/step
how
1/1 [==============================] - 0s 37ms/step
work
1/1 [==============================] - 0s 40ms/step
work
1/1 [==============================] - 0s 39ms/step
work
1/1 [==============================] - 0s 41ms/step
work
1/1 [==============================] - 0s 29ms/step
work
1/1 [==============================] - 0s 31ms/

1/1 [==============================] - 0s 30ms/step
NoSign
1/1 [==============================] - 0s 31ms/step
NoSign
1/1 [==============================] - 0s 32ms/step
NoSign


In [20]:
cap.release()
cv2.destroyAllWindows()